In [6]:
from kfp import dsl
from kfp import compiler


# Ignore FutureWarnings in kfp
import warnings
warnings.filterwarnings("ignore", 
                        category=FutureWarning, 
                        module='kfp.*')

In [12]:
from kfp import local

# Initialize local runner
local.init(runner=local.DockerRunner())


In [13]:
@dsl.component
def say_hello(name: str) -> str:
    hello_text = f'Hello, {name}!'
    
    return hello_text

In [17]:
hello_task = say_hello(name="Shaghayegh")
print(hello_task.output)

19:59:16.704 - INFO - Executing task 'say-hello'
19:59:16.706 - INFO - Streamed logs:

    Found image 'python:3.9'

    [KFP Executor 2025-10-21 23:59:19,110 INFO]: Looking for component `say_hello` in --component_module_path `/tmp/tmp.1P91X8K5dS/ephemeral_component.py`
    [KFP Executor 2025-10-21 23:59:19,110 INFO]: Loading KFP component "say_hello" from /tmp/tmp.1P91X8K5dS/ephemeral_component.py (directory "/tmp/tmp.1P91X8K5dS" and module name "ephemeral_component")
    [KFP Executor 2025-10-21 23:59:19,111 INFO]: Got executor_input:
    {
        "inputs": {
            "parameterValues": {
                "name": "Shaghayegh"
            }
        },
        "outputs": {
            "parameters": {
                "Output": {
                    "outputFile": "/Users/shaghayeghkhalighiyan/local_outputs/say-hello-2025-10-21-19-59-16-703292/say-hello/Output"
                }
            },
            "outputFile": "/Users/shaghayeghkhalighiyan/local_outputs/say-hello-2025-10-21-1

In [18]:
@dsl.component
def how_are_you(hello_task : str) -> str:
    how_are_you = f"{hello_task}. How are you?"
    return how_are_you

In [19]:
how_task = how_are_you(hello_task = hello_task.output)

20:02:26.929 - INFO - Executing task 'how-are-you'
20:02:26.930 - INFO - Streamed logs:

    Found image 'python:3.9'

    [KFP Executor 2025-10-22 00:02:29,542 INFO]: Looking for component `how_are_you` in --component_module_path `/tmp/tmp.dyZrJPOuQX/ephemeral_component.py`
    [KFP Executor 2025-10-22 00:02:29,542 INFO]: Loading KFP component "how_are_you" from /tmp/tmp.dyZrJPOuQX/ephemeral_component.py (directory "/tmp/tmp.dyZrJPOuQX" and module name "ephemeral_component")
    [KFP Executor 2025-10-22 00:02:29,542 INFO]: Got executor_input:
    {
        "inputs": {
            "parameterValues": {
                "hello_task": "Hello, Shaghayegh!"
            }
        },
        "outputs": {
            "parameters": {
                "Output": {
                    "outputFile": "/Users/shaghayeghkhalighiyan/local_outputs/how-are-you-2025-10-21-20-02-26-928355/how-are-you/Output"
                }
            },
            "outputFile": "/Users/shaghayeghkhalighiyan/local_output

In [20]:
print(how_task)
print(how_task.output)

Hello, Shaghayegh!. How are you?


In [23]:
@dsl.pipeline
def hello_pipeline(name : str) -> str:
    hello_task = say_hello(name=name)
    how_task = how_are_you(hello_task = hello_task.output)
    return how_task.output

In [25]:
pipeline_output = hello_pipeline(name="Shaghayegh")
print(pipeline_output)

20:48:23.408 - INFO - Running pipeline: 'hello-pipeline'
--------------------------------------------------------------------------------
20:48:23.410 - INFO - Executing task 'say-hello'
20:48:23.411 - INFO - Streamed logs:

    Found image 'python:3.9'

    [KFP Executor 2025-10-22 00:48:27,591 INFO]: Looking for component `say_hello` in --component_module_path `/tmp/tmp.YRPqzNcJx8/ephemeral_component.py`
    [KFP Executor 2025-10-22 00:48:27,591 INFO]: Loading KFP component "say_hello" from /tmp/tmp.YRPqzNcJx8/ephemeral_component.py (directory "/tmp/tmp.YRPqzNcJx8" and module name "ephemeral_component")
    [KFP Executor 2025-10-22 00:48:27,591 INFO]: Got executor_input:
    {
        "inputs": {
            "parameterValues": {
                "name": "Shaghayegh"
            }
        },
        "outputs": {
            "parameters": {
                "Output": {
                    "outputFile": "/Users/shaghayeghkhalighiyan/local_outputs/hello-pipeline-2025-10-21-20-48-23-407895/

In [26]:
compiler.Compiler().compile(hello_pipeline, 'pipeline.yaml')

In [29]:
pipeline_arguments = {
    "name": "World!",
}

In [30]:
!cat pipeline.yaml

# PIPELINE DEFINITION
# Name: hello-pipeline
# Inputs:
#    name: str
# Outputs:
#    Output: str
components:
  comp-how-are-you:
    executorLabel: exec-how-are-you
    inputDefinitions:
      parameters:
        hello_task:
          parameterType: STRING
    outputDefinitions:
      parameters:
        Output:
          parameterType: STRING
  comp-say-hello:
    executorLabel: exec-say-hello
    inputDefinitions:
      parameters:
        name:
          parameterType: STRING
    outputDefinitions:
      parameters:
        Output:
          parameterType: STRING
deploymentSpec:
  executors:
    exec-how-are-you:
      container:
        args:
        - --executor_input
        - '{{$}}'
        - --function_to_execute
        - how_are_you
        command:
        - sh
        - -c
        - "\nif ! [ -x \"$(command -v pip)\" ]; then\n    python3 -m ensurepip ||\
          \ python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1\
        